In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=3edc9e65f1ada7fa5e24b31363f0f29d5c047eb286b7129a4d2b4ca674e49a70
  Stored in directory: /Users/jlh/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("cumulative.csv")
# Drop the null columns where all values are null
df = df.drop(columns=["kepler_name"])


In [3]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()


In [4]:
df = df.drop(columns=["rowid", "kepid", "koi_pdisposition", "koi_score", "koi_tce_delivname"])


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
#selected_features = df[['koi_disposition', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_steff', 'koi_slogg', 'koi_srad']]
#selected_features
X = df.drop(columns=["koi_disposition", "kepoi_name"], axis=1)

In [6]:
#new selected features

X = df.drop(columns=["koi_disposition", "kepoi_name", "koi_srad_err1", 'koi_srad', 'koi_tce_plnt_num', 'koi_prad_err2', 'koi_srad_err2', 'koi_slogg_err2', 'koi_slogg_err1', 'koi_slogg', 'koi_steff', 'koi_kepmag'], axis=1)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
y = df["koi_disposition"]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
X_train.head()
y_train.head()

7556         CANDIDATE
2272         CANDIDATE
5020         CONFIRMED
7434    FALSE POSITIVE
5058         CANDIDATE
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
label_encoder.fit(y_test)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [11]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
hot_y_train = to_categorical(encoded_y_train)
hot_y_test = to_categorical(encoded_y_test)
hot_y_train

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [11]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=150)
rf = rf.fit(X_train_scaled, hot_y_train)
rf.score(X_test_scaled, hot_y_test)

0.8713480266529985

In [13]:
model2 = rf
model2

RandomForestClassifier(n_estimators=150)

In [14]:
feature_names = X.columns

In [15]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.1180764534445892, 'koi_fpflag_co'),
 (0.07118081318171199, 'koi_fpflag_ss'),
 (0.06572569709573672, 'koi_fpflag_nt'),
 (0.059770092737884244, 'koi_prad'),
 (0.058392304103123904, 'koi_model_snr'),
 (0.043178552612614204, 'koi_fpflag_ec'),
 (0.03510822393452649, 'koi_prad_err2'),
 (0.03265511514788153, 'koi_steff_err1'),
 (0.03247019909274485, 'koi_prad_err1'),
 (0.03211807088589738, 'koi_duration_err2'),
 (0.0296782702333079, 'koi_steff_err2'),
 (0.028482742457759128, 'koi_duration_err1'),
 (0.02519532594201196, 'koi_duration'),
 (0.021875411879524867, 'koi_time0bk_err1'),
 (0.021523042201601925, 'koi_time0bk_err2'),
 (0.020830077584818373, 'koi_depth'),
 (0.020773235748284297, 'koi_impact'),
 (0.020432970440208694, 'koi_period'),
 (0.0195391804503086, 'koi_insol_err1'),
 (0.01756686303433695, 'koi_period_err2'),
 (0.015598617555179758, 'koi_period_err1'),
 (0.015184993216831557, 'koi_insol_err2'),
 (0.015031823506725167, 'koi_insol'),
 (0.014632498744579149, 'koi_teq'),
 (0.013874

In [ ]:
#remove least important columns. Copy and paste to above code and re-run the program
#X = df.drop(columns=["koi_disposition", "kepoi_name", "koi_srad_err1", 'koi_srad', 'koi_tce_plnt_num', 'koi_prad_err2', 'koi_srad_err2', 'koi_slogg_err2', 'koi_slogg_err1', 'koi_slogg', 'koi_steff', 'koi_kepmag'], axis=1)

# Tune the Model



In [22]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [75, 250, 300, 350],
    'max_features': ['auto', 'sqrt', 'log2']
}
CV_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
CV_rf.fit(X_train_scaled, y_train)
print(CV_rf.best_params_)
print(CV_rf.best_score_)

{'max_features': 'auto', 'n_estimators': 250}
0.8795286372229156


In [ ]:
predictions = CV_rf.predict(X_test_scaled)

In [ ]:
CV_rf.score(X_test_scaled, hot_y_test)

# Save the Model

In [23]:
import joblib
filename = 'RandomForest.sav'
joblib.dump(rf, filename)

['RandomForest.sav']